In [9]:
import pandas as pd     #(version 1.0.0)
import plotly           #(version 4.5.4) pip install plotly==4.5.4
import plotly.express as px
import plotly.graph_objs as go

import dash             #(version 1.9.1) pip install dash==1.9.1
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import datetime as dt

app = dash.Dash(__name__)

In [10]:
df = pd.read_csv("patients-maroc.csv")


In [11]:
colors = {
    'background': '#ECF0F1',
    'text': '#33FF63'
}

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184 entries, 0 to 1183
Data columns (total 17 columns):
n                   1184 non-null int64
sex                 64 non-null object
age                 39 non-null float64
country             10 non-null object
province            1184 non-null object
disease             10 non-null float64
group               0 non-null float64
exposure_start      12 non-null object
exposure_end        15 non-null object
infection_reason    1025 non-null object
infection_order     1 non-null float64
infected_by         0 non-null float64
contact_number      0 non-null float64
confirmed_date      1184 non-null object
released_date       6 non-null object
deceased_date       11 non-null object
state               1184 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 157.4+ KB


In [13]:
# too much missing values : print(df.isnull().sum())
df.drop(["sex","age","country","disease","group","infection_order","exposure_start","exposure_end","infected_by","contact_number"],axis = 1,inplace = True)


In [14]:
df['infection_reason'].value_counts()

Local       976
Imported     31
local        13
imported      5
Name: infection_reason, dtype: int64

In [15]:
# replace the missing values of infection_reason with an Unknown reason and standerise the reason format
df.infection_reason.fillna("Unknown", inplace = True)
df.replace(to_replace = ["local"], value = "Local", inplace = True)
df.replace(to_replace = ["imported"], value = "Imported", inplace = True)
infection_reasons = df.infection_reason.value_counts()


In [16]:
#converting to the right type : print(df.dtypes)
df["confirmed_date"] =pd.to_datetime(df.confirmed_date)
df["released_date"] =pd.to_datetime(df.released_date)
df["deceased_date"] =pd.to_datetime(df.deceased_date)

In [17]:
# if nothing is declared in a given day note it as 0
cases_daily = df.confirmed_date.value_counts()
cases_daily = cases_daily.sort_index()



In [18]:
deaths_daily = df.deceased_date.value_counts()
deaths_daily =deaths_daily.sort_index()

In [19]:
releases_daily = df.released_date.value_counts()
releases_daily=releases_daily.sort_index()

In [20]:
# count of days
day_num = (cases_daily.index)[-1]-(cases_daily.index)[0]


In [21]:
days = pd.date_range((cases_daily.index)[0], (cases_daily.index)[-1], freq="D")

In [22]:
for i in days:
    if i not in cases_daily.index:
        cases_daily = cases_daily.append(pd.Series([0], index=[i]))
    if i not in deaths_daily.index:
        deaths_daily = deaths_daily.append(pd.Series([0], index=[i]))
    if i not in releases_daily.index:
        releases_daily = releases_daily.append(pd.Series([0], index=[i]))


In [23]:
#sort everything
cases_daily = cases_daily.sort_index()
deaths_daily = deaths_daily.sort_index()
releases_daily = releases_daily.sort_index()


In [24]:
#accumulate valuesfor i in cases_daily.index:
sum_cases = 0
accumulate_cases = cases_daily.copy()

sum_deaths = 0
accumulate_deaths = deaths_daily.copy()

sum_releases = 0
accumulate_releases = releases_daily.copy()

for i in days:
    sum_cases = sum_cases + cases_daily[i]
    accumulate_cases[i] = sum_cases
    sum_deaths = sum_deaths + deaths_daily[i]
    accumulate_deaths[i] = sum_deaths
    sum_releases = sum_releases + releases_daily[i]
    accumulate_releases[i] = sum_releases

In [25]:
# generate daily traces
trace_cases_daily = go.Scatter(x = list(cases_daily.index),
                               y = list(cases_daily.values),
                               name = "cases_daily",
                               mode= "lines",
                               line = dict(color = "#0852a1"))
trace_deaths_daily = go.Scatter(x = list(deaths_daily.index),
                               y = list(deaths_daily.values),
                               name = "death_daily",
                               mode= "lines",
                               line = dict(color = "#d40b0b"))
trace_releases_daily = go.Scatter(x = list(releases_daily.index),
                               y = list(releases_daily.values),
                               name = "releases_daily",
                               mode= "lines",
                               line = dict(color = "#1aa108"))

In [26]:
# generate accumulate traces
trace_accumulate_cases = go.Scatter(x = list(accumulate_cases.index),
                                    y = list(accumulate_cases.values),
                                    name = "accumulate_cases",
                                    mode= "lines",
                                    line = dict(color = "#0852a1"))
trace_accumulate_deaths = go.Scatter(x = list(accumulate_deaths.index),
                                    y = list(accumulate_deaths.values),
                                    name = "accumulate_deaths",
                                    mode= "lines",
                                    line = dict(color = "#d40b0b"))
trace_accumulate_releases = go.Scatter(x = list(accumulate_releases.index),
                                    y = list(accumulate_releases.values),
                                    name = "accumulate_releases",
                                    mode= "lines",
                                    line = dict(color = "#1aa108"))

In [27]:
#do nut chart
trace_infection_reasons = go.Pie(labels = infection_reasons.index,
                                 values = infection_reasons.values,
                                 hole=.5)

In [28]:
# construct a data of recordes per province
df_sub = pd.DataFrame(df[df.state == "isolated"].province.value_counts())
df_sub.columns = ["isolated"]
lon = [-7.965636,-8.365910,-6.235980,-4.658982,-5.433289,-2.556609,-5.330057,-6.267116,-8.634435,-13.178729,-9.875821]
lat = [33.134399,31.736601,34.140537,33.726610,35.197405,33.991792,31.258876,32.492824,29.992446,27.131740,28.581917]
deceased = df[df.state == "Deceased"].province.value_counts()
exit = df[df.state == "Exit"].province.value_counts()
df_sub["lon"] = lon
df_sub["lat"] = lat
df_sub["deceased"] = deceased
df_sub["exit"] = exit
df_sub.fillna(0, inplace=True)

In [29]:
#traces for a geo plot
trace_isolated_geo = go.Scattergeo(
    name = "isolated",
    locationmode = 'country names',
    lon = df_sub.lon,
    lat = df_sub.lat,
    text = df_sub.index,
    marker = dict(
        sizeref=.1,
        size = df_sub.isolated,
        sizemode = "area",
    ),
    hovertemplate=
        "<b>%{text}</b><br>" +
        "Number of: %{marker.size}<br>" +
        "<extra></extra>",
)

In [30]:
trace_deceased_geo = go.Scattergeo(
    name = "deceased",
    locationmode = 'country names',
    lon = df_sub.lon,
    lat = df_sub.lat,
    text = df_sub.index,
    marker = dict(
        sizeref=.1,
        size = df_sub.deceased,
        sizemode = "area",
    ),
    hovertemplate=
        "<b>%{text}</b><br>" +
        "Number of: %{marker.size}<br>" +
        "<extra></extra>",
)

In [31]:
trace_exit_geo = go.Scattergeo(
    name = "exit",
    locationmode = 'country names',
    lon = df_sub.lon,
    lat = df_sub.lat,
    text = df_sub.index,
    marker = dict(
        sizeref=.1,
        size = df_sub.exit,
        sizemode = "area"
    ),
    hovertemplate=
        "<b>%{text}</b><br>" +
        "Number of: %{marker.size}<br>" +
        "<extra></extra>",
)

In [32]:
#traces for a bar chart
trace_isolated = go.Bar(
    x = df_sub.index,
    y = df_sub.isolated,
    name = "isolated"
)
trace_deceased = go.Bar(
    x = df_sub.index,
    y = df_sub.deceased,
    name = "deceased"
)
trace_exit = go.Bar(
    x = df_sub.index,
    y = df_sub.exit,
    name = "exit"
)

In [33]:
print((df.state == "isolated") & (df.confirmed_date <= days[5]))

0       False
1       False
2       False
3       False
4       False
        ...  
1179    False
1180    False
1181    False
1182    False
1183    False
Length: 1184, dtype: bool


In [34]:
colors = {
    'background': '#ECF0F1',
    'text': '#33FF63'
}

In [42]:
app = dash.Dash(__name__)

app.layout = html.Div([
    
    html.H1(children='COVID-19',
           style={
               'textAlign':'center',
               'color':'#FF3333'}),
    
     html.H3(children=
        'Dash: patients-maroc.',
             style={
               'textAlign':'center',
               'color':colors['text'] }),
    html.Br(),
    
    
     html.Div([
        dcc.Graph(figure ={"data":[trace_cases_daily,trace_deaths_daily,trace_releases_daily],
                         "layout":{"title":"Number of recorded covid-19 cases dayly"}
                         },
        className='nine columns'),

        html.Br(),
        html.Div([
            html.H3(["Accumulate Cases"]),
            dcc.Graph(
                figure ={"data":[trace_accumulate_cases,trace_accumulate_deaths,trace_accumulate_releases],
                         "layout":{"title":"Number of recorded covid-19 until each day"}
                         },
            ),
        ], className="col-6"),
    ], className="row"),
        html.Br(),

    
    
     html.Div([
        dcc.Graph(
            figure={
                "data": [trace_infection_reasons],
                "layout":{"title":"Infection reasons",
                          "scope": 'africa'},

            }
        )
    ]),
    html.Br(),
    
    
    html.Div([
        dcc.Graph(
            id = "geo_fig",
            figure={
                "data": [trace_isolated_geo,trace_deceased_geo,trace_exit_geo],
                "layout":{"title":"Map view",
                          "geo":{"resolution":50,
                                 "showframe": False,
                                 "showland" : True,"framewidth": 1000,
                                 "landcolor" : "rgb(217, 217, 217)",
                                 "fitbounds": "locations",
                                 "showcoastlines":True, "coastlinecolor":"White",
                                 "showcountries": True,"countrycolor": "White"},
                          "hoverinfo": "text",
                          }
            }
        )
    ]),
    html.Br(),
    
    html.Div([
        dcc.Graph(
            id = "bar_chart",
            figure ={
                "data": [trace_isolated,trace_deceased,trace_exit],
                "layout": {"title": "Bar Chart"}
            }
        )
    ]),
    html.Div([
        dcc.Slider(
            id="date_selector",
            min=0,
            max=(df.confirmed_date.max()-df.confirmed_date.min()).days,
            step = 1,
            value=(df.confirmed_date.max()-df.confirmed_date.min()).days,
        )
    ]),
    html.Br(),

        html.H3(id = "date_selected"),
        html.Label(['Choose column:'],style={'font-weight': 'bold', "text-align": "center"}),
])

        

In [40]:
#callback to update the day displayed
@app.callback(
    dash.dependencies.Output("date_selected", "children"),
    [dash.dependencies.Input("date_selector", "value")])
def update_date_text(value):
    print(value)
    return '{}'.format(days.strftime("%d %b %Y")[value])


#callback to update the bar chart
@app.callback(
    dash.dependencies.Output("bar_chart", "figure"),
    [dash.dependencies.Input("date_selector", "value")])
def update_bar_chart(value):
    isolated = df[(df.state == "isolated") & (df.confirmed_date <= days[value])].province.value_counts()
    deceased = df[(df.state == "Deceased") & (df.confirmed_date <= days[value])].province.value_counts()
    exit = df[(df.state == "Exit") & (df.confirmed_date <= days[value])].province.value_counts()
    df_sub["isolated"] = isolated
    df_sub["deceased"] = deceased
    df_sub["exit"] = exit
    df_sub.fillna(0, inplace=True)
    print(df_sub)
    # traces for a bar chart
    trace_isolated = go.Bar(
        x=df_sub.index,
        y=df_sub.isolated,
        name="isolated"
    )
    trace_deceased = go.Bar(
        x=df_sub.index,
        y=df_sub.deceased,
        name="deceased"
    )
    trace_exit = go.Bar(
        x=df_sub.index,
        y=df_sub.exit,
        name="exit"
    )
    return {"data": [trace_isolated,trace_deceased,trace_exit],
            "layout": {"title": "Bar Chart"}}



#update the geo fig
@app.callback(
    dash.dependencies.Output("geo_fig", "figure"),
    [dash.dependencies.Input("date_selector", "value")])
def update_geo_fig(value):
    isolated = df[(df.state == "isolated") & (df.confirmed_date <= days[value])].province.value_counts()
    deceased = df[(df.state == "Deceased") & (df.confirmed_date <= days[value])].province.value_counts()
    exit = df[(df.state == "Exit") & (df.confirmed_date <= days[value])].province.value_counts()
    df_sub["isolated"] = isolated
    df_sub["deceased"] = deceased
    df_sub["exit"] = exit
    df_sub.fillna(0, inplace=True)
    print(df_sub)
    # traces for a bar chart
    trace_isolated_geo = go.Scattergeo(
        name = "isolated",
        locationmode='country names',
        lon=df_sub.lon,
        lat=df_sub.lat,
        text=df_sub.index,
        marker=dict(
            sizeref=.1,
            size=df_sub.isolated,
            sizemode="area",
        ),
        hovertemplate=
        "<b>%{text}</b><br>" +
        "Number of: %{marker.size}<br>" +
        "<extra></extra>",
    )
    trace_deceased_geo = go.Scattergeo(
        name="deceased",
        locationmode='country names',
        lon=df_sub.lon,
        lat=df_sub.lat,
        text=df_sub.index,
        marker=dict(
            sizeref=.1,
            size=df_sub.deceased,
            sizemode="area",
        ),
        hovertemplate=
        "<b>%{text}</b><br>" +
        "Number of: %{marker.size}<br>" +
        "<extra></extra>",
    )
    trace_exit_geo = go.Scattergeo(
        name="exit",
        locationmode='country names',
        lon=df_sub.lon,
        lat=df_sub.lat,
        text=df_sub.index,
        marker=dict(
            sizeref=.1,
            size=df_sub.exit,
            sizemode="area"
        ),
        hovertemplate=
        "<b>%{text}</b><br>" +
        "Number of: %{marker.size}<br>" +
        "<extra></extra>",
    )
    return {"data": [trace_isolated_geo,trace_deceased_geo,trace_exit_geo],
            "layout":{"title":"Was testing",
                      "geo":{"resolution":50,
                             "showframe": False,
                             "showland" : True,"framewidth": 1000,
                             "landcolor" : "rgb(217, 217, 217)",
                             "fitbounds": "locations",
                             "showcoastlines":True, "coastlinecolor":"White",
                             "showcountries": True,"countrycolor": "White"},
                      "hoverinfo": "text",}}

In [41]:
if __name__ == '__main__':
    app.run_server(port=8050)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/May/2020 15:04:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:04:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:04:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:04:51] "GET /_dash-component-suites/dash_core_components/async-dropdown.v1_9_1m1586525729.js HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:04:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:04:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:04:51] "POST /_dash-update-component HTTP/1.1" 200 -


36
                                isolated        lon        lat  deceased  exit
Casablanca - Settat                  349  -7.965636  33.134399       2.0   2.0
Marrakech - Safi                     197  -8.365910  31.736601       4.0   3.0
Rabat - Sale - Kenitra               185  -6.235980  34.140537       1.0   0.0
Fes - Meknes                         155  -4.658982  33.726610       4.0   1.0
Tanger - Tetouan - Al Houceima        98  -5.433289  35.197405       0.0   1.0
Oriental                              73  -2.556609  33.991792       0.0   0.0
Daraa - Tafilalet                     51  -5.330057  31.258876       0.0   0.0
Beni Mellal - Khenifra                32  -6.267116  32.492824       1.0   1.0
Souss - Massa                         19  -8.634435  29.992446       0.0   0.0
Laayoune - Saguia al hamra             4 -13.178729  27.131740       0.0   0.0
Guelmim - Oued Noun                    1  -9.875821  28.581917       0.0   0.0
                                isolated        l

127.0.0.1 - - [11/May/2020 15:05:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:05:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:05:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:05:44] "GET /_favicon.ico?v=1.11.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2020 15:05:44] "POST /_dash-update-component HTTP/1.1" 200 -


36
                                isolated        lon        lat  deceased  exit
Casablanca - Settat                  349  -7.965636  33.134399       2.0   2.0
Marrakech - Safi                     197  -8.365910  31.736601       4.0   3.0
Rabat - Sale - Kenitra               185  -6.235980  34.140537       1.0   0.0
Fes - Meknes                         155  -4.658982  33.726610       4.0   1.0
Tanger - Tetouan - Al Houceima        98  -5.433289  35.197405       0.0   1.0
Oriental                              73  -2.556609  33.991792       0.0   0.0
Daraa - Tafilalet                     51  -5.330057  31.258876       0.0   0.0
Beni Mellal - Khenifra                32  -6.267116  32.492824       1.0   1.0
Souss - Massa                         19  -8.634435  29.992446       0.0   0.0
Laayoune - Saguia al hamra             4 -13.178729  27.131740       0.0   0.0
Guelmim - Oued Noun                    1  -9.875821  28.581917       0.0   0.0
                                isolated        l